# SNEWPY Demo

## Getting Started

### Resources

* [snewpy repository on GitHub](https://github.com/SNEWS2/snewpy)
* [snewpy documentation](https://snewpy.rtfd.io)
* snewpy papers:
  * Software review: [![DOI](https://joss.theoj.org/papers/10.21105/joss.03772/status.svg)](https://doi.org/10.21105/joss.03772)
  * Underlying physics: [arXiv:2109.08188](https://arxiv.org/abs/2109.08188) or [DOI:10.3847/1538-4357/ac350f](https://dx.doi.org/10.3847/1538-4357/ac350f)

### Installation
<!-- Click the following button to open this notebook in Binder. Everything should just work.
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/JostMigenda/PyHEP2022/main?labpath=snewpy_demo.ipynb)-->

To run this notebook locally on your own machine, use `pip install snewpy==1.3` to install snewpy and `git clone https://github.com/SNOwGLoBES/snowglobes.git; cd snowglobes; git checkout v1.2` to download [SNOwGLoBES](https://github.com/SNOwGLoBES/snowglobes), which we will need in the second half of this notebook to estimate event rates in realistic detectors.

### Initialising Example Models

For this demo, we will use neutrino fluxes from `Nakazato_2013` and `Zha_2021`, but plenty of other supernova models are available as part of snewpy.
Let’s see what models are available as part of each model family!

In [ ]:
import snewpy
snewpy.model_path = "./SNEWPY_models/"  # download directory for model files
from snewpy.models.ccsn import Nakazato_2013, Zha_2021
from astropy import units as u

print("Zha models:\n", Zha_2021.param)
print("Nakazato models:\n", Nakazato_2013.param)
# Note: For the Nakazato model, not all parameter combinations are valid. Use `get_param_combinations()` to view and filter valid combinations:
#highz_models = list(params for params in Nakazato_2013.get_param_combinations() if params['metallicity'] == 0.02)
#print("Progenitors with metallicity z=0.02:", *highz_models, sep='\n')

Let’s pick one progenitor from each model family. If this is the first time you’re using a progenitor, snewpy will automatically download the required data files for you.

If you look at the data files, you’ll notice that these model families use very different input file formats, different time binning, etc. Thankfully, snewpy takes care of all of that and provides us with a unified interface.

In [ ]:
zha = Zha_2021(progenitor_mass=17*u.solMass)
nakazato = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=200*u.ms, metallicity=0.004, eos='shen')

nakazato

## Plotting Different Supernova Models


In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from snewpy.neutrino import Flavor

mpl.rc('font', size=16)

# Plot luminosity of both models
fig, ax = plt.subplots(1, figsize=(10, 6))

for flavor in Flavor:
    ax.plot(zha.time, zha.luminosity[flavor]/1e51,  # Report luminosity in units foe/s
            label=flavor.to_tex() + ' (Zha)',
            color='C0' if flavor.is_electron else 'C2',
            ls='-' if flavor.is_neutrino else '--',
            lw=2)

# for flavor in Flavor:
#     ax.plot(nakazato.time, nakazato.luminosity[flavor]/1e51,  # Report luminosity in units foe/s
#             label=flavor.to_tex() + ' (Nakazato)',
#             color='C1' if flavor.is_electron else 'C3',
#             ls='-' if flavor.is_neutrino else '--',
#             lw=1)

ax.set(xlim=(-0.05, 0.8), xlabel=r'$t-t_{\rm bounce}$ [s]', ylabel=r'luminosity [$10^{51}$ erg s$^{-1}$]')
ax.grid()
ax.legend(loc='upper right', ncol=2, fontsize=18);

## Applying Neutrino Flavo(u)r Transformations

Supernova simulations only provide us with the neutrino fluxes *emitted near the centre of the supernovae*. The fluxes *observed in a detector on Earth* can be very different.

The module `snewpy.flavor_transformation` contains many different transformations that can happen along the way. Here, we will use the `AdiabaticMSW` transformation, which occurs as the neutrinos exit the star. (In brief: The electron density changes between the centre of the star and the surface, which modifies the effective mass of electron (anti-)neutrinos; whereas muon or tau (anti-)neutrinos are unaffected.)

Other flavor transformations involving e.g. non-adiabatic MSW, decoherence, neutrino decay or sterile neutrinos are also available.

In [ ]:
from snewpy.flavor_transformation import AdiabaticMSW
from snewpy.neutrino import Flavor, MassHierarchy

from astropy import units as u
import matplotlib as mpl
import numpy as np

def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino fluxes."""
    
    energies = np.linspace(0,60,121) * u.MeV
    
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    
    # Compute the transformed and untransformed flux at each time.
    ispec = model.get_flux(times, energies, 10*u.kpc)
    tspec_nmo = model.get_flux(times, energies, 10*u.kpc, xform_nmo)
    tspec_imo = model.get_flux(times, energies, 10*u.kpc, xform_imo)

    ilum = {}
    tlum_nmo = {}
    tlum_imo = {}
    # integrate over energy to get the flux
    for flavor in Flavor:
        ilum[flavor] = np.trapz(ispec[flavor].to('1/(cm**2 *erg*s)'), energies.to('erg')).value 
        tlum_nmo[flavor] = np.trapz(tspec_nmo[flavor].to('1/(cm**2 *erg*s)'), energies.to('erg')).value
        tlum_imo[flavor] = np.trapz(tspec_imo[flavor].to('1/(cm**2 *erg*s)'), energies.to('erg')).value            

    # make the figures 
    fig, axes = plt.subplots(3,3, figsize=(20,12), tight_layout=True)
    
    smax = [0.,0.,0.]
    titles = ['Untransformed', 'Transformed (NMO)', 'Transformed (IMO)']
    for i, spec in enumerate([ilum, tlum_nmo, tlum_imo]):
        for j, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            ax = axes[i,j]
            timeunits = 'ms' if j==0 else 's'
            
            for flavor in Flavor:
                if i == 0:
                    smax[j] = np.maximum(smax[j], 1.1*np.max(spec[flavor][phase]))
                
                ax.plot(times[phase].to(timeunits),
                        spec[flavor][phase], label=flavor.to_tex(), lw=3,
                        color='C0' if flavor.is_electron else 'C1',
                        ls='-' if flavor.is_neutrino else ':')
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[j]))
            
            if j==0:
               ax.set(ylabel=r'flux [cm$^{-2}$ s$^{-1}$]')
               ax.legend(loc='upper right', ncol=1, fontsize=18)
            if j==1:
                ax.set(title=titles[i])
            if i < 2:
                ax.set(xticklabels=[])
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
            
            ax.grid(ls=':')

    return fig

def plot_spectra(model, xform, t):
    """Plot initial and oscillated neutrino spectra at a fixed time."""

    energies = np.linspace(0,60,121) * u.MeV   

    #get the spectra
    ispec = model.get_initial_spectra(t, energies) 
    tspec = model.get_transformed_spectra(t, energies, xform)

    fig, axes = plt.subplots(1, 2, figsize=(18,7), sharey=True, tight_layout=True)

    for i, spec in enumerate([ispec, tspec]):
        axes[0].plot(energies, spec[Flavor.NU_E].to('1/(MeV*s)'),
                    label='Untransformed '+Flavor.NU_E.to_tex() if i==0 else 'Transformed '+Flavor.NU_E.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[0].plot(energies, spec[Flavor.NU_X].to('1/(MeV*s)'), 
                    label='Untransformed '+Flavor.NU_X.to_tex() if i==0 else 'Transformed '+Flavor.NU_X.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[0].set(xlabel=r'$E$ [{}]'.format(energies.unit), title='Neutrino spectra at $t = ${:.1f}'.format(t))
        axes[0].grid()
        axes[0].legend(loc='upper right', ncol=2, fontsize=16)
    
        axes[1].plot(energies, spec[Flavor.NU_E_BAR].to('1/(MeV*s)'), 
                    label='Untransformed '+Flavor.NU_E_BAR.to_tex() if i==0 else 'Transformed '+Flavor.NU_E_BAR.to_tex(),
                    color='C0', ls='-' if i==0 else ':', lw=2,  alpha=0.7)
        axes[1].plot(energies, spec[Flavor.NU_X_BAR].to('1/(MeV*s)'), 
                    label='Untransformed '+Flavor.NU_X_BAR.to_tex() if i==0 else 'Transformed '+Flavor.NU_X_BAR.to_tex(),
                    color='C1', ls='-' if i==0 else ':', lw=2,  alpha=0.7)

        axes[1].set(xlabel=r'$E$ [{}]'.format(energies.unit), title='Antineutrino spectra at $t = ${:.1f}'.format(t)) 
        axes[1].grid()
        axes[1].legend(loc='upper right', ncol=2, fontsize=16)    
    
    ax = axes[0]
    ax.set(ylabel=r'number luminosity [MeV$^{-1}$ s$^{-1}$]')
    
    return fig

After defining two helper functions above, let’s use the first one to plot the neutrino flux over time! Focus on the “neutronization burst” in the left panel—this will show most clearly how some of the electron neutrinos are transformed into muon/tau neutrinos:

(Note: Most supernova simulations don’t distinguish between muon & tau neutrinos, since their interactions are virtually identical; so we simply label them as NU_X here.)

In [ ]:
xform_nmo = AdiabaticMSW()
xform_imo = AdiabaticMSW(mh=MassHierarchy.INVERTED)

fig = plot_total_flux(nakazato, xform_nmo, xform_imo)
# fig.savefig('flux_adiabaticmsw.pdf')

Next, we’ll use the second helper function to plot the neutrino spectra at a fixed point in time. Note how the total flux of electron (anti-) neutrinos decreases due to the flavour transformation, but their mean energy increases.

In [ ]:
fig_nmo = plot_spectra(nakazato, xform_nmo, 100*u.ms)
# fig_imo = plot_spectra(nakazato, xform_imo, 100*u.ms)

## Using SNEWPY as a SNOwGLoBES Front End

Finally, we can use snewpy to calculate observed spectra or event rates for our models in various neutrino detectors.
This requires SNOwGLoBES, a separate software package that contains cross sections for all relevant interaction channels and files describing energy smearing & efficiency for many detectors.

### Observed Event Spectra for Different Interaction Channels

Let’s plot the observed event spectra for different interaction channels.
First, we pick a detector, supernova model and time range:

In [ ]:
from snewpy import snowglobes

SNOwGLoBES_path = "./snowglobes/"  # where SNOwGLoBES is located

# set distance in kpc
distance = 10

# Set detector. You can use any detector configuration available in SNOwGLoBES, for example:
detector = "wc100kt30prct"  # like Super- or Hyper-Kamiokande
# detector = "ar40kt"  # like DUNE

# set SNEWPY model type and filename
modeltype = 'Zha_2021'
model = 's17.dat'

# set desired flavor transformation
transformation = 'AdiabaticMSW_NMO' # 'NoTransformation'

# Construct file system path of model file and name of output file
# The output file will be stored in the same directory as the model file.
modelfile = f"{snewpy.model_path}/{modeltype}/{model}"
outfile = f"{modeltype}_{model}_{transformation}"

# There are multiple ways to select a time range.
# Option 1 - don't specify tstart and tend, then the whole model is integrated
# tstart = None
# tend = None

# Option 2 - specify single tstart and tend, this makes 1 fluence file integrated over the window
tstart = 0.0 * u.s
tend = 0.3 * u.s

Next, we perform the calculation in three steps:
1. `generate_fluence` integrates the flux over the specified time window(s) to get the fluence as a function of energy
2. `simulate` multiplies the fluence, cross section and detector smearing & efficiency to get the number of observed events as a function of energy
3. `collate` collects the results for different channels into a single dictionary

In [ ]:
print("Preparing fluences ...")
tarredfile = snowglobes.generate_fluence(modelfile, modeltype, transformation, distance, outfile, tstart, tend)

print("Calculating events ...")
snowglobes.simulate(SNOwGLoBES_path, tarredfile, detector_input=detector)

print("Collating results ...")
tables = snowglobes.collate(SNOwGLoBES_path, tarredfile, skip_plots=True)

In [ ]:
tables[f"Collated_{outfile}_{detector}_events_smeared_weighted.dat"]
# SNOwGLoBES also provides “unsmeared” events, without accounting for detector effects.

In [ ]:
key = f"Collated_{outfile}_{detector}_events_smeared_weighted.dat"
channels = tables[key]['header'].split()
energies = tables[key]['data'][0] * 1e3  # convert GeV to MeV
events = {}
for i, channel in enumerate(channels):
    if i==0: continue  # skip "Energy" array
    events[channel] = tables[key]['data'][i]
events['total'] = sum(tables[key]['data'][1:])

for channel in ['total'] + channels[1:]:
    plt.plot(energies, events[channel], label=channel)

plt.xlabel("Energy [MeV]")
plt.ylabel("Events per energy bin")
plt.yscale("log")
plt.xlim(0, 70)
plt.ylim(bottom=0.3)
plt.legend(loc="upper right")
plt.savefig(f'plot_{outfile}_{detector}.pdf')

print(f"Total Events: {sum(events['total']):.3f}")
print(f"Mean Detected Energy: {sum(energies * events['total']) / sum(events['total']):.3f} MeV")

### Observed Event Rate

Let’s plot the observable event rate in a realistic detector. Specifically, let’s try to reproduce the event rate shown in Figure 5 of [Zha et al. (2021)](https://arxiv.org/pdf/2103.02268.pdf).

First, we select the same detector, flavor transformation and time range:

In [ ]:
from snewpy import snowglobes

SNOwGLoBES_path = "./snowglobes/"  # where SNOwGLoBES is located

# set distance in kpc
distance = 10

# Set detector. You can use any detector configuration available in SNOwGLoBES, for example:
detector = "icecube"

# set SNEWPY model type and filename
modeltype = 'Zha_2021'
model = 's17.dat'

# set desired flavor transformation
transformation = 'AdiabaticMSW_NMO'

# Construct file system path of model file and name of output file
# The output file will be stored in the same directory as the model file.
modelfile = f"{snewpy.model_path}/{modeltype}/{model}"
outfile = f"{modeltype}_{model}_{transformation}"

# There are multiple ways to select a time range. Here, we will use
# Option 3 = specify sequence of time intervals, one fluence file is made for each interval
window_tstart = 0.742
window_tend = 0.762
n_bins = 60  # use 120 for better time resolution (but we don’t want to wait as long during the live demo)
tstart = np.linspace(window_tstart, window_tend, n_bins, endpoint=False) * u.s
tend = tstart + (window_tend - window_tstart) / n_bins * u.s

Next, we perform the calculation in three steps:
1. `generate_fluence` integrates the flux over the specified time window(s) to get the fluence as a function of energy
2. `simulate` multiplies the fluence, cross section and detector smearing & efficiency to get the number of observed events as a function of energy
3. `collate` collects the results for different channels into a single dictionary

In [ ]:
print("Preparing fluences ...")
tarredfile = snowglobes.generate_fluence(modelfile, modeltype, transformation, distance, outfile, tstart, tend)

print("Calculating events ...")
snowglobes.simulate(SNOwGLoBES_path, tarredfile, detector_input=detector)

print("Collating results ...")
tables = snowglobes.collate(SNOwGLoBES_path, tarredfile, skip_plots=True)

In [ ]:
nevents = np.zeros(n_bins)
for i in range(n_bins):
    key = f"Collated_{outfile}_{i}_{detector}_events_smeared_weighted.dat"
    
    # Sum events across all interaction channels and energy bins
    for j in range(1,len(tables[key]['header'].split())):
        nevents[i] += sum(tables[key]['data'][j])

# nevents is per bin, convert to per ms
factor = n_bins / (window_tend - window_tstart) / 1000

plt.plot(tstart - 0.742*u.s, nevents * factor, color="C2")
plt.xlabel("$t-t_{2c}$ [s]")
plt.ylabel("Counts [ms$^{-1}$]")
plt.ylim(0, 2001);
# compare to Figure 5 of Zha et al. (2021)

## Summary and Q&A

* snewpy provides …
    * … a unified way to access 100s of SN models
    * … many different flavor transformations between neutrino production and detection
    * … an easy way to estimate event rates in neutrino detectors
* SN models and flavor transformations can be used by other software
* New SN models or flavor transformations are welcome!

* **Interested? Have questions?**
    * E-mail me at jost.migenda@kcl.ac.uk
    * Or [open an issue on GitHub](https://github.com/SNEWS2/snewpy/issues)

